In [27]:
from cnn import model
from importlib import reload
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

In [61]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [79]:
import cnn
reload(cnn.model)
from cnn import model

num_classes = 7
learning_rate = 0.02
model = model.CnnChordClassifier(num_classes)
model = model.to(device)
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [10]:
original_dataset = ImageFolder(root='data/extracted_chords')

id2chord = {0:'A',
            1:'B',
            2:'C',
            3:'D',
            4:'E',
            5:'F',
            6:'G'
           }

In [26]:
### augmentation
rotation_transform = transforms.Compose([
    transforms.RandomRotation(30)
])

rotate_crop_transform = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomCrop((240, 180))
])

color_transform = transforms.Compose([
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
])

rotate_crop_color_transform = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomCrop((240, 180)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
])

rotate_grayscale_transform = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.Grayscale(),
    
])


for image, label in original_dataset:
    for i in range(10):
        augmented_image = rotate_grayscale_transform(image)
        augmented_image.save(f'data/extracted_chords/{id2chord[label]}/rotated_grayScaled_image_{i}.jpg')


In [82]:
transform = transforms.Compose([
    transforms.Resize((320, 240)),
    transforms.ToTensor()
])

# Create a dataset from image folder
dataset = ImageFolder(root='data/extracted_chords', transform=transform)

# Define the proportions for train, val, and test sets
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Calculate the sizes of each subset based on the ratios
train_size = int(train_ratio * len(dataset))
val_size = int(val_ratio * len(dataset))
test_size = len(dataset) - train_size - val_size

# Randomly split the dataset into train, val, and test subsets
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create data loaders for each subset
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [83]:
# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    # Print the average loss for each epoch
    print(f"Epoch {epoch+1}/{num_epochs} Loss: {running_loss / len(train_loader)}")

Epoch 1/100 Loss: 1.9768568575382233
Epoch 2/100 Loss: 1.9252843260765076
Epoch 3/100 Loss: 1.9108319580554962
Epoch 4/100 Loss: 1.8825411200523376
Epoch 5/100 Loss: 1.850725769996643
Epoch 6/100 Loss: 1.7912158071994781
Epoch 7/100 Loss: 1.8530492186546326
Epoch 8/100 Loss: 1.7529355883598328
Epoch 9/100 Loss: 1.6985299587249756
Epoch 10/100 Loss: 1.7857278883457184
Epoch 11/100 Loss: 1.7376114428043365
Epoch 12/100 Loss: 1.4505001306533813
Epoch 13/100 Loss: 1.7613718807697296
Epoch 14/100 Loss: 1.5843869745731354
Epoch 15/100 Loss: 1.3419923186302185
Epoch 16/100 Loss: 1.3853447437286377
Epoch 17/100 Loss: 1.3303861320018768
Epoch 18/100 Loss: 1.604103296995163
Epoch 19/100 Loss: 1.289755403995514
Epoch 20/100 Loss: 1.308532327413559
Epoch 21/100 Loss: 1.1457333862781525
Epoch 22/100 Loss: 1.0540392994880676
Epoch 23/100 Loss: 1.1146150678396225
Epoch 24/100 Loss: 0.9394915103912354
Epoch 25/100 Loss: 1.160257175564766
Epoch 26/100 Loss: 0.9266520589590073
Epoch 27/100 Loss: 0.72501

In [84]:
# Evaluate the model
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Validation Accuracy: {accuracy}")

Validation Accuracy: 0.7735849056603774
